In [38]:
import pandas as pd
import numpy as np
import dill
from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.preprocessing import OrdinalEncoder
from sklearn.ensemble import GradientBoostingRegressor
import warnings
warnings.filterwarnings("ignore")

### Загрузка данных

Датасет с данными об автомобилях<br>
https://www.kaggle.com/amanabdullayev/used-cars<br>
Задача регрессии - предсказать цену автомобиля

In [39]:
df = pd.read_csv('car_data.csv')
df

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Mileage,RegistrationMonth,FuelType,Brand,NotRepaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
0,24/03/2016 11:52,480,NaN,1993,manual,0,golf,150000,0,petrol,volkswagen,NaN,24/03/2016 00:00,0,70435,07/04/2016 03:16
1,24/03/2016 10:58,18300,coupe,2011,manual,190,NaN,125000,5,gasoline,audi,yes,24/03/2016 00:00,0,66954,07/04/2016 01:46
2,14/03/2016 12:52,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,NaN,14/03/2016 00:00,0,90480,05/04/2016 12:47
3,17/03/2016 16:54,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no,17/03/2016 00:00,0,91074,17/03/2016 17:40
4,31/03/2016 17:25,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no,31/03/2016 00:00,0,60437,06/04/2016 10:17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
354364,21/03/2016 09:50,0,NaN,2005,manual,0,colt,150000,7,petrol,mitsubishi,yes,21/03/2016 00:00,0,2694,21/03/2016 10:42
354365,14/03/2016 17:48,2200,NaN,2005,NaN,0,NaN,20000,1,NaN,sonstige_autos,NaN,14/03/2016 00:00,0,39576,06/04/2016 00:46
354366,05/03/2016 19:56,1199,convertible,2000,auto,101,fortwo,125000,3,petrol,smart,no,05/03/2016 00:00,0,26135,11/03/2016 18:17
354367,19/03/2016 18:57,9200,bus,1996,manual,102,transporter,150000,3,gasoline,volkswagen,no,19/03/2016 00:00,0,87439,07/04/2016 07:15


In [40]:
df.shape

(354369, 16)

In [41]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   DateCrawled        354369 non-null  object
 1   Price              354369 non-null  int64 
 2   VehicleType        316879 non-null  object
 3   RegistrationYear   354369 non-null  int64 
 4   Gearbox            334536 non-null  object
 5   Power              354369 non-null  int64 
 6   Model              334664 non-null  object
 7   Mileage            354369 non-null  int64 
 8   RegistrationMonth  354369 non-null  int64 
 9   FuelType           321474 non-null  object
 10  Brand              354369 non-null  object
 11  NotRepaired        283215 non-null  object
 12  DateCreated        354369 non-null  object
 13  NumberOfPictures   354369 non-null  int64 
 14  PostalCode         354369 non-null  int64 
 15  LastSeen           354369 non-null  object
dtypes: int64(7), object(

### Обзор признаков

Выделим вещественные и категориальные переменные и целевую переменную<br>
В качестве вещественных возьмём **Power** (Мощность двигателя), **Mileage** (Пробег)<br>
В качестве категориальных переменных можно использовать **Gearbox** (Тип коробки передач) и **FuelType** (Тип топлива)<br>
Целевая переменная - **Price** (Цена автомобиля)

In [42]:
num_features = ['Power', 'Mileage']
cat_features = ['Gearbox', 'FuelType']
target = 'Price'

In [43]:
df.describe()

,Price,RegistrationYear,Power,Mileage,RegistrationMonth,NumberOfPictures,PostalCode
count,354369.000000,354369.000000,354369.000000,354369.000000,354369.000000,354369.0,354369.000000
mean,4416.656776,2004.234448,110.094337,128211.172535,5.714645,0.0,50508.689087
std,4514.158514,90.227958,189.850405,37905.341530,3.726421,0.0,25783.096248
min,0.000000,1000.000000,0.000000,5000.000000,0.000000,0.0,1067.000000
25%,1050.000000,1999.000000,69.000000,125000.000000,3.000000,0.0,30165.000000
50%,2700.000000,2003.000000,105.000000,150000.000000,6.000000,0.0,49413.000000
75%,6400.000000,2008.000000,143.000000,150000.000000,9.000000,0.0,71083.000000
max,20000.000000,9999.000000,20000.000000,150000.000000,12.000000,0.0,99998.000000


#### Power

In [44]:
df['Power'].value_counts()

0       40225
75      24023
60      15897
150     14590
101     13298
        ...  
610         1
6006        1
5815        1
6018        1
1241        1
Name: Power, Length: 712, dtype: int64

Нулевые значения нужно заменить на среднее

#### Mileage

In [45]:
df['Mileage'].value_counts()

150000    238209
125000     36454
100000     14882
90000      11567
80000      10047
70000       8593
60000       7444
5000        6397
50000       6232
40000       4911
30000       4436
20000       3975
10000       1222
Name: Mileage, dtype: int64

#### Gearbox

In [46]:
df['Gearbox'].value_counts()

manual    268251
auto       66285
Name: Gearbox, dtype: int64

В данном признаке 2 значения, которые нужно закодировать числами

#### FuelType

In [47]:
df['FuelType'].value_counts()

petrol      216352
gasoline     98720
lpg           5310
cng            565
hybrid         233
other          204
electric        90
Name: FuelType, dtype: int64

В данном признаке 7 значений, которые нужно закодировать числами

#### Пропуски

In [48]:
df[cat_features].isna().sum()

Gearbox     19833
FuelType    32895
dtype: int64

Пропуски в категориальных признаках заполним наиболее часто встречающимися значениями

### Разделение данных на test и train

In [49]:
X_train, X_test, y_train, y_test = train_test_split(df, df['Price'], test_size=0.33, random_state=42)
X_test.to_csv("X_test.csv", index=None)
y_test.to_csv("y_test.csv", index=None)
X_train.to_csv("X_train.csv", index=None)
y_train.to_csv("y_train.csv", index=None)

### Построение pipeline

In [50]:
class ColumnSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[self.key]

class NumberSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    Use on numeric columns in the data
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[[self.key]]        
    
class ValueImputer(BaseEstimator, TransformerMixin):
    def __init__(self, key, value):
        self.key = key
        self.value = value
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        X[self.key] = X[self.key].replace(self.value, X[self.key].mean())
        return X

class Imputer(BaseEstimator, TransformerMixin):
    def __init__(self, key, value):
        self.key = key
        self.value = value
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        X[self.key] = X[self.key].fillna(self.value)
        return X

In [51]:
Power = Pipeline([('selector', NumberSelector(key = 'Power')),
                  ('imputer', ValueImputer('Power', 0)),
                  ('scaler', StandardScaler())
                  ])

Mileage = Pipeline([('selector', NumberSelector(key = 'Mileage')),
                    ('scaler', StandardScaler())
                  ])

Gearbox = Pipeline([('selector', NumberSelector(key = 'Gearbox')),
                  ('imputer', Imputer('Gearbox', 'manual')),
                  ('encoder', OrdinalEncoder())
                  ])

FuelType = Pipeline([('selector', NumberSelector(key = 'FuelType')),
                  ('imputer', Imputer('FuelType', 'petrol')),
                  ('encoder', OrdinalEncoder())
                  ])


In [52]:
features = FeatureUnion([('Power', Power),
                      ('Mileage', Mileage),
                      ('Gearbox', Gearbox),
                      ('FuelType', FuelType)])
feature_processing = Pipeline([('features', features)])

### Обучение модели

In [53]:
pipeline = Pipeline([
    ('features',features),
    ('regressor', GradientBoostingRegressor(max_depth=20, min_samples_leaf=30, n_estimators=50, random_state=42)),
])
pipeline.fit(X_train, y_train)

Pipeline(steps=[('features',
                 FeatureUnion(transformer_list=[('Power',
                                                 Pipeline(steps=[('selector',
                                                                  NumberSelector(key='Power')),
                                                                 ('imputer',
                                                                  ValueImputer(key='Power',
                                                                               value=0)),
                                                                 ('scaler',
                                                                  StandardScaler())])),
                                                ('Mileage',
                                                 Pipeline(steps=[('selector',
                                                                  NumberSelector(key='Mileage')),
                                                                 ('scaler',
    

### Сохранение модели в файл

In [54]:
with open('model_reg.dill', 'wb') as f:
  dill.dump(pipeline, f)